In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from langchain_chroma import Chroma
import chromadb

from dotenv import load_dotenv
import os

from PyPDF2 import PdfReader

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()


True

Creating the embeddings

In [3]:
embeddings= GoogleGenerativeAIEmbeddings(
    # model="gemini-embedding-001"
    model="models/gemini-embedding-exp-03-07",
    
)

E0000 00:00:1759254529.826769 9021555 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Creating the LLM

In [4]:
llm= ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.3,
    max_tokens=500,
    timeout=None,
    max_retries=2,
    api_key= os.getenv("GOOGLE_API_KEY")
)

E0000 00:00:1759254529.840464 9021555 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Get the pdf and extracting the text

In [5]:
pdfPath= "/Users/consultadd/Prajwal/LangChain/RAG/documents/cars_info.pdf"
reader= PdfReader(pdfPath)

pdfContent= [page.extract_text() for page in reader.pages]

In [6]:
print(pdfContent)

['Information about Cars\n \nCars are one of the most widely used modes of transportation in the modern world. They run\nprimarily on internal combustion engines, though electric cars have gained immense popularity in\nrecent years due to environmental concerns and advances in battery technology. Types of Cars: 1.\nSedan - Comfortable passenger cars with separate compartments for engine, passengers, and\nluggage. 2. SUV (Sport Utility Vehicle) - Larger, more powerful cars, often with off-road capabilities.\n3. Hatchback - Compact cars with a rear door that swings upward, providing access to the cargo\narea. 4. Coupe - Stylish two-door cars, often sporty in nature. 5. Convertible - Cars with roofs that\ncan be retracted or removed. 6. Electric Cars - Run on electric motors powered by rechargeable\nbatteries. 7. Hybrid Cars - Combine internal combustion engines with electric motors for better fuel\nefficiency. Key Components of Cars: - Engine: The heart of the car, traditionally powered 

Splitter

In [7]:
splitters= RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap= 100    
)

chunks= splitters.create_documents(pdfContent)

Now storing the chunks(in the form of Dcoument) into the vector database

In [8]:
dbPath= os.path.join(os.getcwd(), 'pdfDB')

client= chromadb.Client(
    settings=chromadb.config.Settings(is_persistent= True, persist_directory= dbPath)
)

collections= client.get_or_create_collection("Pdfs")

In [9]:
collections.add(
    ids= [f"doc_{i}" for i in range(len(chunks))],
    documents= [i.page_content for i in chunks],
    embeddings= embeddings.embed_documents([i.page_content for i in chunks]),
)

Now making the retrieval to get the relevant documents

In [10]:
vectorestore= Chroma(
    client= client,
    collection_name="Pdfs",
    embedding_function=embeddings
)

retriever= vectorestore.as_retriever(
    search_type= "mmr",
    search_kwargs= {"k": 3, "fetch_k": 10}
)

Making the parser

In [11]:
class RAGAnswer(BaseModel):
    mode: str = Field(description="Either 'context+reasoning' or 'reasoning'")
    answer: str = Field(description="The answer to the question")
    
parser= PydanticOutputParser(pydantic_object=RAGAnswer)

Making the Prompt

In [12]:
prompt= PromptTemplate(
    template="""
    You are a helpful assistant. Decide how to answer based on the context:

    - If the provided context is useful, combine it with your reasoning. 
    In that case set "mode" = "context+reasoning".
    - If the context is missing or irrelevant, rely only on your reasoning. 
    In that case set "mode" = "reasoning".
    - Always return valid JSON.

    Context:
    {context}

    Question:
    {question}

    {format_instructions}
    """,
    input_variables= ["context", "question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
    
)

In [13]:
def format_docs(retrieved_docs):
    context_text= "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

parallel_chain= RunnableParallel({
    "question": RunnablePassthrough(),
    "context": retriever | RunnableLambda(format_docs)
})

final_chain= parallel_chain | prompt | llm | parser

In [14]:
query= "Do you know about the Mahindra XUV 3xo?"

# result= final_chain.invoke({"question":query})
result= final_chain.invoke(query)

In [20]:
final_result= result.model_dump()
print(final_result['mode'])
print(final_result['answer'])

reasoning
I do not have specific information about the Mahindra XUV 3xo in my current knowledge base. My information about cars covers general types, components, and some modern innovations, but not details on every specific model.
